In [1]:
!pip install git+https://github.com/shahashka/SP-GIES 

  Cloning https://github.com/shahashka/SP-GIES to /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-pky_jofe
  Running command git clone --filter=blob:none --quiet https://github.com/shahashka/SP-GIES /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-pky_jofe
  Resolved https://github.com/shahashka/SP-GIES to commit c2f3fc0ca5151e1ef7ea9b82bd64f79b342b6603
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
from sp_gies import utils
from sp_gies import sp_gies as sp
import sp_gies
sp_gies.__version__

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


'1.0'

## Generate a random graph and random data.
Data is sampled from a linear Gaussian model (child nodes are weighted sums of parent nodes). Options for random graphs are scale_free, erdos_renyi, and small_world. Choose the number of nodes in the graphs (n), number of observational samples (nsamples) and number of interventional samples per node (iv_samples). p, k are parameters for the random graphs.

Arcs is a list of edge pairs e.g. [("G1","G2"), ("G2","G4")]

In [3]:
(arcs,nodes,bias,var), data = utils.get_random_graph_data("scale_free", n=10, nsamples=10000, iv_samples=10, p=0.5, k=2)

The dataset should have column names corresponding to the random variables names. Additionally a 'target' column specifies which intervention index was performed to obtain the sample. The target column indexes from 1 (for compatability with R). For example in the dataframe below, target=1 indicates that 'G1' was intervened to obtain the corresponding interventional sample. For observational samples the target value should be 0. 


In [4]:
print(data.head)

<bound method NDFrame.head of            G1        G2        G3        G4        G5        G6        G7  \
0    7.463500  1.661807  1.150648  3.017491  1.601714 -0.488938  0.331825   
1    0.201073  0.047384  1.295767  1.631285 -1.383862 -0.740284 -1.813678   
2   14.426114  3.339053  0.726225  3.868996  8.498602  0.286319  1.807987   
3    7.638654  1.907038 -0.316147  3.806496  5.945730  0.891800  2.048113   
4   12.030557  1.949552  0.919539  4.352302  7.782638  0.730665  2.540060   
..        ...       ...       ...       ...       ...       ...       ...   
5    7.002592  1.787334  0.971919  3.337484  4.201892 -0.914213  1.251815   
6    1.029250  1.407253  0.454169  2.343053  6.002986 -0.918451  2.299300   
7   -5.482092 -0.388794  0.711828 -1.116055 -1.230666 -0.768775 -1.504115   
8   10.097164  2.679455  0.452144  4.403974  9.642562  0.106754  5.598606   
9    5.254433  1.775286  1.092521  3.123651  3.845733 -0.171352  1.673384   

          G8         G9       G10  target  
0

# Run the SP-GIES algorithm on the dataset.
Runs the PC algorithm if pc=True and then the GIES algorithm on the output CPDAG.
Optionally provide an input skeleton, which should be a numpy array with number rows/columns corresponding to the number of nodes. The output adjacency matrix is saved in the output dir as sp_gies-adj_mat.csv.

For a 10 node graph this should take less than 1 second

In [5]:
adj_mat = sp.sp_gies(data, outdir="./", skel=None, pc=True)

# Evaluate the estimated graph.
Convert to networkx Digraph and run evaluation metrics. 

SHD is the L1 error between the graphs.

SID is the number of wrong interventional distributions. This method scales quadratically with the number of nodes. For large graphs it can take a while, so there is an option to turn off this evaluation with get_sid=False.

AUC is the error under the precision recall curve

In [6]:
G_true = utils.edge_to_dag(arcs)
G_sp_gies = utils.adj_to_dag(adj_mat, nodes)
print(utils.get_scores(["SP-GIES"], [G_sp_gies], G_true, get_sid=True))

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


SP-GIES SHD: 12.0 SID: 35.0 AUC: 0.6849999999999999
None


adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
